In [1]:
import fitsio

import numpy as np

import astropy.units as u

from astropy.coordinates import EarthLocation
from astropy.coordinates import SkyCoord
from astropy.coordinates import AltAz
from astropy.coordinates import ICRS
from astropy.time import Time

# example usage:
#altaz_to_skycoord(alt=90.0 * u.deg, az=0.0 * u.deg, time=time, location=mylocation)
def altaz_to_skycoord(alt, az, time, location):
    loc = AltAz(alt=alt, az=az, obstime=time, location=location)
    coord = loc.transform_to(ICRS)
    return coord

def fits_read(filename):
    data, hdr = fitsio.read(filename, header=True)
    return data, hdr

def fitsheader_to_location(hdr):
    return EarthLocation(
        lat=hdr['LATITUDE'] * u.deg,
        lon=hdr['LONGITUD'] * u.deg,
        height=hdr['HEIGHT'] * u.m,
    )

def fitsheader_to_time(hdr):
    return Time(hdr['DATE-OBS'], format='fits', scale='utc')

In [2]:
filename = './initial_wcs-ogg-1597473421-night.fits.fz'

In [3]:
data, hdr = fits_read(filename)
location = fitsheader_to_location(hdr)
print('LOCATION:', location)
obstime = fitsheader_to_time(hdr)
print('OBSTIME', obstime)

LOCATION: (-5466087.7314879, -2404265.70005887, 2242156.4309251) m
OBSTIME 2020-08-15T06:37:01.487


In [4]:
from astropy.coordinates import solar_system_ephemeris
from astropy.coordinates import get_body
from astropy.coordinates import ICRS
from astropy.coordinates import AltAz
from astropy.coordinates import Angle

# https://docs.astropy.org/en/stable/coordinates/transforming.html
# https://docs.astropy.org/en/stable/coordinates/solarsystem.html

with solar_system_ephemeris.set('jpl'):
    astropy_jupiter = get_body('jupiter', time=obstime, location=location)
    print('ASTROPY Jupiter:', astropy_jupiter)
    astropy_jupiter_icrs = astropy_jupiter.transform_to(ICRS)
    print('ASTROPY Jupiter ICRS (RA/DEC):', astropy_jupiter_icrs)
    astropy_jupiter_altaz = astropy_jupiter.transform_to(AltAz(location=location, obstime=obstime))
    print('ASTROPY Jupiter AltAz:', astropy_jupiter_altaz)

ASTROPY Jupiter: <SkyCoord (GCRS: obstime=2020-08-15T06:37:01.487, obsgeoloc=(-321734.71862663, -5962571.55176577, 2242788.74778963) m, obsgeovel=(434.79661793, -23.78376417, -0.85763875) m / s): (ra, dec, distance) in (deg, deg, km)
    (289.99315705, -22.55750854, 6.39575814e+08)>
ASTROPY Jupiter ICRS (RA/DEC): <SkyCoord (ICRS): (ra, dec, distance) in (deg, deg, km)
    (296.72047035, -21.49478298, 7.6832872e+08)>
ASTROPY Jupiter AltAz: <SkyCoord (AltAz: obstime=2020-08-15T06:37:01.487, location=(-5466087.7314879, -2404265.70005887, 2242156.4309251) m, pressure=0.0 hPa, temperature=0.0 deg_C, relative_humidity=0.0, obswl=1.0 micron): (az, alt, distance) in (deg, deg, m)
    (151.1064587, 41.22187261, 6.39575814e+11)>


In [5]:
from dateutil.parser import parse
import ephem

observer = ephem.Observer()
observer.lon = np.radians(hdr['LONGITUD'])
observer.lat = np.radians(hdr['LATITUDE'])
observer.elev = hdr['HEIGHT']
observer.date = parse(hdr['DATE-OBS'])

ephem_jupiter = ephem.Jupiter()
ephem_jupiter.compute(observer)

ephem_jupiter_alt, ephem_jupiter_az = Angle(f'{ephem_jupiter.alt} deg'), Angle(f'{ephem_jupiter.az} deg')
ephem_jupiter_altaz = AltAz(alt=ephem_jupiter_alt, az=ephem_jupiter_az, location=location, obstime=obstime)
print('EPHEM Jupiter AltAz:', ephem_jupiter_altaz)
ephem_jupiter_icrs = ephem_jupiter_altaz.transform_to(ICRS)
print('EPHEM Jupiter ICRS (RA/DEC):', ephem_jupiter_icrs)

EPHEM Jupiter AltAz: <AltAz Coordinate (obstime=2020-08-15T06:37:01.487, location=(-5466087.7314879, -2404265.70005887, 2242156.4309251) m, pressure=0.0 hPa, temperature=0.0 deg_C, relative_humidity=0.0, obswl=1.0 micron): (az, alt) in deg
    (151.10725, 41.24033333)>
EPHEM Jupiter ICRS (RA/DEC): <ICRS Coordinate: (ra, dec) in deg
    (289.97781629, -22.54229852)>


In [6]:
from astropy.wcs import WCS

mywcs = WCS(hdr)

In [7]:
# To reiterate:
# Both astropy and ephem get the AltAz correct
print(f'Ephem AltAz: {ephem_jupiter_altaz}')
# And ephem converts to RA/DEC correctly too... (verified by *OPENING THE IMAGE* in DS9)
# and locating Jupiter. Which is centered at approx x=233 y=91
print(f'Ephem RA/DEC: {ephem_jupiter_icrs}')
ephem_x, ephem_y = mywcs.all_world2pix(ephem_jupiter_icrs.ra.value, ephem_jupiter_icrs.dec.value, 1)
print(f'Ephem Calculates Jupiter at x={ephem_x} y={ephem_y}')

Ephem AltAz: <AltAz Coordinate (obstime=2020-08-15T06:37:01.487, location=(-5466087.7314879, -2404265.70005887, 2242156.4309251) m, pressure=0.0 hPa, temperature=0.0 deg_C, relative_humidity=0.0, obswl=1.0 micron): (az, alt) in deg
    (151.10725, 41.24033333)>
Ephem RA/DEC: <ICRS Coordinate: (ra, dec) in deg
    (289.97781629, -22.54229852)>
Ephem Calculates Jupiter at x=233.16781868023395 y=90.11092846066475


In [8]:
# To reiterate:
# Both astropy and ephem get the AltAz correct
print(f'Astropy AltAz: {astropy_jupiter_altaz}')
# But astropy really messes up the conversion to RA/DEC :-(
# 22px in X is a LONG WAYS OFF. Plotting this with lots of data shows that
# astropy calculates the plane of the solar system correctly, but the planets
# are offset from where they should be. To the "left" on this image.
print(f'Astropy RA/DEC: {astropy_jupiter_icrs}')
astropy_x, astropy_y = mywcs.all_world2pix(astropy_jupiter_icrs.ra.value, astropy_jupiter_icrs.dec.value, 1)
print(f'Astropy Calculates Jupiter at x={astropy_x} y={astropy_y}')

Astropy AltAz: <SkyCoord (AltAz: obstime=2020-08-15T06:37:01.487, location=(-5466087.7314879, -2404265.70005887, 2242156.4309251) m, pressure=0.0 hPa, temperature=0.0 deg_C, relative_humidity=0.0, obswl=1.0 micron): (az, alt, distance) in (deg, deg, m)
    (151.1064587, 41.22187261, 6.39575814e+11)>
Astropy RA/DEC: <SkyCoord (ICRS): (ra, dec, distance) in (deg, deg, km)
    (296.72047035, -21.49478298, 7.6832872e+08)>
Astropy Calculates Jupiter at x=209.97078396766184 y=95.11110266992372
